# MoBioFP - Fingerphoto Recognition (Fingertip Object Detection)

In [ ]:
# TODO: Import only the necessary functions from shared.py
from shared import *

## Define global constants

In [ ]:
SAMPLE_DIR = "../data/raw/samples"
MODEL_CHECKPOINT = "MODEL_CHECKPOINT_FILE_PATH"

## Read sample images

In [ ]:
images, images_titles = read_images(SAMPLE_DIR, rotate=True, rotate_angle=90)

In [ ]:
show_images(images, images_titles, fig_size=15, sup_title="Sample Fingerphoto Images")

## Fingertip detection using YOLOv8n pre-trained model

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL_CHECKPOINT)
model.info()
results = model(images, stream=True, conf=0.80, max_det=1)

predicted_images = []
bbox_coords = []
fingertip_images = []

for result in results:
    boxes = result.boxes.xyxy.tolist()
    boxes = [int(coord) for coord in boxes[0]]
    bbox_coords.append(boxes)

    original = result.orig_img
    x1, y1, x2, y2 = boxes
    fingertip = original[y1:y2, x1:x2]
    fingertip_images.append(fingertip)

    predicted = result.plot()
    predicted_images.append(predicted)

show_images(predicted_images, images_titles, fig_size=15, sup_title="YOLOv8n Fingertip Detection")

In [ ]:
show_images(fingertip_images, images_titles, sup_title="Fingertip Images")

In [ ]:
from rembg import remove, new_session
from mobiofp.utils import post_process_mask

# Initialize the rembg session
rembg_session = new_session("isnet-general-use")

# Remove background from the cropped images
fingertip_masks = [
    remove(fingertip, only_mask=True, session=rembg_session) for fingertip in fingertip_images
]

# Post-process masks
fingertip_masks = [post_process_mask(mask) for mask in fingertip_masks]

show_images(fingertip_masks, images_titles, sup_title="Fingertip Masks")

## Fingertip Image-Quality Assessment

In [ ]:
from mobiofp.utils import quality_scores

sharpness_scores = []
contrast_scores = []
mask_coverage_scores = []

for image, mask in zip(fingertip_images, fingertip_masks):
    sharpness_score, contrast_score, mask_coverage_scorere = quality_scores(image, mask)
    sharpness_scores.append(sharpness_score)
    contrast_scores.append(contrast_score)
    mask_coverage_scores.append(mask_coverage_scorere)

show_iqa(sharpness_scores, contrast_scores, mask_coverage_scores)

In [ ]:
print(f"Number of images before filtering: {len(fingertip_images)}")
print(f"Number of masks before filtering: {len(fingertip_masks)}")

# NOTE:
#
# For the scope of this demonstration, we will consider only the binary mask coverage score
# and we will not consider the sharpness and contrast scores. The binary mask coverage threshold
# is set to 70% only to include all the images in the dataset since the dataset is already too small
# and we want to demonstrate the next steps of the pipeline.
BINARY_MASK_COVERAGE_THRESH = 70.0

iqa_images = []
iqa_masks = []
iqa_titles = []

for mcs, fingertip, fingertip_mask, fingertip_title in zip(
    mask_coverage_scores, fingertip_images, fingertip_masks, images_titles
):
    if mcs >= BINARY_MASK_COVERAGE_THRESH:
        iqa_images.append(fingertip)
        iqa_masks.append(fingertip_mask)
        iqa_titles.append(fingertip_title)

assert len(iqa_images) == len(iqa_masks) == len(iqa_titles)

print(f"Number of images after filtering: {len(iqa_images)}")
print(f"Number of masks after filtering: {len(iqa_masks)}")

## Fingertip Enhancement

In [ ]:
grayscale_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in iqa_images]
grayscale_images = [
    cv2.bitwise_and(image, image, mask=mask) for image, mask in zip(grayscale_images, iqa_masks)
]

show_images(grayscale_images, iqa_titles, cmap="gray", sup_title="Grayscale Fingertip Images")

In [ ]:
from mobiofp.utils import fingertip_enhancement

fingertip_enhanced_images = [fingertip_enhancement(image) for image in grayscale_images]

show_images(fingertip_enhanced_images, iqa_titles, sup_title="Fingertip Enhanced Images")

## Fingertip to Fingeprint Conversion

The function `fingerprint_mapping()` takes an fingertip-enhanced and converts it into a fingerprint image.
It does this by:

- Resizing the image.
- Calculating the local gradient of the image using Sobel filters.
- Calculating the orientation of the ridges in the fingerprint.
- Extracting a region of the image and smoothing it to reduce noise.
- Calculating the x-signature of the region and finding its local maxima to estimate the ridge period.
- Creating a bank of Gabor filters with different orientations.
- Filtering the image with each filter in the bank.
- Assembling the final result by taking the corresponding convolution result for each pixel based on the closest orientation in the Gabor bank.
- Converting the result to grayscale.

In [ ]:
from mobiofp.utils import fingerprint_mapping

fingerprints, fingerprint_titles = [], []

for image, title in zip(fingertip_enhanced_images, iqa_titles):
    fingerprint = fingerprint_mapping(image)
    if fingerprint is not None:
        fingerprints.append(fingerprint)
        fingerprint_titles.append(title)

show_images(fingerprints, fingerprint_titles, sup_title="Fingerprint Images", fig_size=15)

In [ ]:
from mobiofp.utils import fingerprint_enhancement

# Apply fingerprint enhancement using Fingerprint-Enhancement-Python package
#
# Ref: https://github.com/Utkarsh-Deshmukh/Fingerprint-Enhancement-Python
#
# NOTE: This step is one of the slowest steps in the end-to-end pipeline for our fingerphoto matching algorithm
fingerprint_enhanced_images = [fingerprint_enhancement(fingerprint) for fingerprint in fingerprints]

show_images(
    fingerprint_enhanced_images, iqa_titles, sup_title="Fingerprint Enhanced Images", fig_size=15
)

## Save Fingerprint Images

In [ ]:
PROCESSED_DIR = Path("../data/processed/samples/detection")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

for image, title in zip(fingerprint_enhanced_images, fingerprint_titles):
    fingerprint_path = PROCESSED_DIR / f"{title}.png"
    cv2.imwrite(str(fingerprint_path), image)

## Probe vs Gallery

In [ ]:
probe_images, probe_titles = zip(
    *[
        (image, title)
        for image, title in zip(fingerprint_enhanced_images, fingerprint_titles)
        if "n" in title
    ]
)
gallery_images, gallery_titles = zip(
    *[
        (image, title)
        for image, title in zip(fingerprint_enhanced_images, fingerprint_titles)
        if "w" in title
    ]
)

# Skeletonize the fingerprint images
probe_images = [cv2.ximgproc.thinning(image) for image in probe_images]
gallery_images = [cv2.ximgproc.thinning(image) for image in gallery_images]

## Feature Extraction - ORB (Oriented FAST and Rotated BRIEF)

In [ ]:
from mobiofp.utils import imkpts

# Initialize the ORB detector
orb = cv2.ORB_create()

# Find the keypoints and descriptors for the probe images
probe_orb_keypoints, probe_orb_descriptors = zip(
    *[orb.detectAndCompute(image, None) for image in probe_images]
)

# Show keypoints for each probe image
probe_orb_keypoints_images = [
    imkpts(image, keypoints, cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    for image, keypoints in zip(probe_images, probe_orb_keypoints)
]

# Show keypoints for each probe image
show_images(
    probe_orb_keypoints_images,
    probe_titles,
    fig_size=20,
    sup_title="Probe ORB Keypoints",
)

# Find the keypoints and descriptors for the gallery images
gallery_orb_keypoints, gallery_orb_descriptors = zip(
    *[orb.detectAndCompute(image, None) for image in gallery_images]
)

# Show keypoints for each gallery image
gallery_orb_keypoints_images = [
    imkpts(image, keypoints, cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    for image, keypoints in zip(gallery_images, gallery_orb_keypoints)
]

# Show keypoints for each probe image
show_images(
    gallery_orb_keypoints_images,
    gallery_titles,
    fig_size=20,
    sup_title="Gallery ORB Keypoints",
)

## Feature Matching - FLANN based matcher

In [ ]:
import numpy as np

from mobiofp.utils import orb_flann_matcher

distance_matrix = {}

for p_img, p_title, p_kpts, p_desc in zip(
    probe_images, probe_titles, probe_orb_keypoints, probe_orb_descriptors
):
    distance_matrix[p_title] = {}

    for g_img, g_title, g_kpts, g_desc in zip(
        gallery_images, gallery_titles, gallery_orb_keypoints, gallery_orb_descriptors
    ):
        values = {}
        distance, matches_img = orb_flann_matcher(
            p_img, p_kpts, p_desc, g_img, g_kpts, g_desc, include_img=True
        )
        if np.isnan(distance) or np.isinf(distance):
            distance = -1
        values["distance"] = distance
        values["matches_img"] = matches_img
        distance_matrix[p_title][g_title] = values

In [ ]:
matched_images = []
matched_images_titles = []

for p_title in distance_matrix:
    for g_title in distance_matrix[p_title]:
        p_subject, p_illumination, p_finger, p_background, _ = p_title.split("_")
        g_subject, g_illumination, g_finger, g_background, _ = g_title.split("_")

        if p_subject == g_subject and p_finger == g_finger:
            matched_images.append(distance_matrix[p_title][g_title]["matches_img"])
            title = f"Probe: ({p_subject}, {p_illumination}, {p_finger}, {p_background}) = {distance_matrix[p_title][g_title]['distance']:.2f}"
            matched_images_titles.append(
                f"{p_title} vs {g_title} = {distance_matrix[p_title][g_title]['distance']:.2f}"
            )

plt.figure(figsize=(20, 20))
for i, (image, title) in enumerate(zip(matched_images, matched_images_titles), 1):
    plt.suptitle("Matched Image (distances)", fontsize=24)
    plt.subplot(2, 2, i)
    plt.imshow(image)
    plt.title(title, fontsize=16)
    plt.tight_layout()
    plt.axis("off")
plt.show()